# SETUP

In [ ]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain_google_genai  \
  langchain-community==0.3.16 \
  pydantic==2.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 66.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [ ]:
import pandas as pd
import numpy as np
import re
import time
from collections import defaultdict

import json
from google.colab import userdata
import os
from datetime import datetime


from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.prompts import ChatPromptTemplate

In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
google_model = 'gemini-2.5-flash-lite'

llm = ChatGoogleGenerativeAI(model=google_model, temperature=0.0)

In [ ]:
# Define the input file path
input_file = '/content/drive/MyDrive/parsed_reports.json'

# Load the JSON data from the file
with open(input_file, 'r') as f:
    json_data = json.load(f)

# Preprocessing

In [ ]:
# Dictionaries

danger_word_ranking = {"Al Qaeda":10,"explosives":9,"training camp":7,"bomb":7,"Semtex":9,"C-4":9,"e-bomb":8,"chemical weapons":8,
                       "Taliban":7,"SCADA systems":7,"jihad":6,"Hamas":6,"Hezbollah":6,"Islamic Jihad":6,"FARC":5,"ELN":5,
                       "weapons":6,"terrorism":6,"militants":5,"insurgent":5,"assault":4,"attack":4,"threat":4,
                       "danger":4,"security":4,"surveillance":4,"intelligence":4,"suspected":3,"suspect":3,"arrested":4,
                       "apprehended":3,"wanted":3,"kidnapping":3,"hostage":3,"sabotage":3,"assassination":3,"plot":3,"conspiracy":3,
                       "extremist":3,"radical":3,"militia":4,"organization":3,"network":3,"cells":3,"operatives":3,"agents":3,"courier":3,
                       "funds":3,"financing":3,"money laundering":3,"trafficking":3,"narcotics":3,"diamonds":3,
                       "weapons of mass destruction":3,"biological weapons":3,"nuclear":3,"dirty bomb":3,"radiological":3,"cyber attack":3,
                       "infrastructure":3,"critical infrastructure":3,"airport":3,"embassy":3,"bank":3,"casino":3,"mosque":3,"university":3,
                       "passport":3,"visa":3,"driver's license":3,"forged":3,"fraudulent":3,"alias":3,"fake":3,"document":3,"information":3,
                       "intercepted":3,"e-mail":3,"phone call":3,"communication":3,"informant":3,"source":3,"meeting":3,"gathering":3,
                       "residence":3,"safe house":3,"hideout":3,"abandoned":3,"vehicle":3,"car rental":3,"flight":3,"ticket":3,"travel":3,
                       "border crossing":3,"smuggled":3,"illegal":3,"entry":3,"exit":3,"detained":3,"custody":3,"interrogation":3,
                       "confiscated":3,"seized":3,"raid":3,"search warrant":3,"evidence":3,"investigation":3,"analysis":3,"report":3,
                       "bulletin":3,"alert":3,"warning":3,"concern":3,"vulnerability":3,"risk":3,"potential":3,"possible":3,"activity":3,
                       "operation":3,"mission":3,"objective":3,"goal":3,"plan":3,"scheme":3,"intent":3,"purpose":3,"motivation":3,
                       "ideology":3,"propaganda":3,"recruitment":3,"radicalization":3,"extremism":3,"fanaticism":3,"zealot":3,
                       "fanatic":3,"terrorist":3,"militant":3,"sleeper cell":3,"sleeper agent":3,"undercover":3,"asset":3,"foot and mouth disease":4}

location_importance_rank = {
    'Los Palmas Hotel/Calamar/Columbia': 3, 'Cuba': 1, 'Peshawar/Pakistan': 6, 'Detroit/Michigan/USA': 7, 'Texas/USA': 7, 'Columbia/Missouri/USA': 5,
    'Kuala Lumpur/Malaysia': 6, 'Freeport/Bahamas': 4, 'Paris/France': 10, 'Coral Gables/Florida/USA': 5, 'Potomac/Maryland/USA': 4,
    'the Isle of Capri Casino/Kansas City/Missouri/USA': 3, 'Santo Domingo/Dominican Republic': 5, 'Bahamas': 1, 'Sears Tower/Chicago/Illinois/USA': 7,
    'Clifton/New Jersey/USA': 5, 'London/UK': 10, 'Madrid/Spain': 8, 'Piedras Negras/Mexico': 4, 'Miami/Florida/USA': 7,'Houston/Texas/USA': 7,
    'Afghanistan': 1, 'UK': 1, "O'Hare Airport/Chicago/Illinois/USA": 7, 'Islamabad/Pakistan': 6, 'Bucaramanga/Columbia': 5, 'Marrakech/Morocco': 5,
    'Russia': 1, 'Arlington/Texas/USA': 6, 'Feyzabad/Afghanistan': 4,
    'Grand Prairie/Texas/USA': 4, 'Savannah/Georgia/USA': 4, 'Montreal/Quebec/Canada': 6,
    'Jersey National Bank/Newark/New Jersey/USA': 3, 'The Galleria Shopping Mall/Houston/Texas/USA': 3,
    'Chicago/Illinois/USA': 7, 'Ciudad Acuna/Mexico': 4, 'Amsterdam/Netherlands': 3,
    'Iran': 1, 'New Brunswick/New Jersey/USA': 3, 'Dubai/UAE': 7,
    'Karachi/Pakistan': 7, 'Cabrito Hotel/Ojinaga/Mexico': 3, 'Uvalde/Texas/USA': 3,
    'Herndon/Virginia/USA': 3, 'Moroccan Oasis Hotel/Casablanca/Marocco': 3,
    'Miami Beach/Florida/USA': 4, 'Santa Domingo/Dominican Republic': 5,
    'Denton/Texas/USA': 4, 'Jeddah': 6, 'River Rouge Park/Detroit/Michigan/USA': 3,
    'Windsor/Ontario/Canada': 5, 'Victoria/Texas/USA': 3, 'Hyderabad/Pakistan': 4,
    'Culver City/California/USA': 4, 'Pakistan': 1, 'Calamar/Columbia': 4,
    'Carrizo Springs/Texas/USA': 3, 'Gizab': 1, 'Sahara Atlantic Hotel/Casablanca/Marocco': 3,
    'Dulles International Airport/Dulles/Virginia/USA': 7, 'New York City/New York/USA': 10,
    'Nassau/Bahamas': 4, 'Seashore Hotel/Nassau/Bahamas': 3, 'Islamabad': 6,
    'Norfolk/Virginia/USA': 3, 'Tampico/Mexico': 4, 'Havana/Cuba': 5,
    'Kirov/Russia': 3, 'Los Angeles/California/USA': 8, 'Bellaire/Texas/USA': 4,
    'Buffalo/New York/USA': 5, "Harrah's Casino/Kansas City/Missouri/USA": 3,
    'Casablanca/Marocco': 5, 'Cairo/Egypt': 8, 'Mineral Wells/Texas/USA': 2,
    'Bogota/Columbia': 5, 'Ann Arbor/Michigan/USA': 3, 'Parachinar/Pakistan': 3,
    'Saudi Arabia': 1, 'Tampa/Florida/USA': 5, 'Panama City/Panama': 6,
    'Netherlands': 1, 'Toronto/Canada': 6, 'Moroccan Sands Hotel/Casablanca/Marocco': 3,
    'Brooklyn/New York/USA': 5, 'Plum Island': 1, 'Fort Lauderdale/Florida/USA': 4,
    'Qalat': 2, 'Dallas/Texas/USA': 6, 'CDG/Paris/France': 3,
    'Lebanon': 1, 'Pakhtia/Afghanistan': 2, 'Chechnya/Russia': 3,
    'Toronto Airport/Toronto/Canada': 5, 'Cartagena/Columbia': 5,
    'Caribbean': 1, 'Los Andes Hotel/Bogota/Columbia': 3, 'Mexico': 1,
    'Laurel/Maryland/USA': 3, 'Cuidad Acuna/Mexico': 3, 'Inglewood/California/USA': 3,
    'Peru': 1, 'Santo Domingo/The Dominican Republic': 5, 'Freeport/Bahamas': 3,
    'Caracas/Venezuela': 5, 'San Antonio/Texas/USA': 5, 'Antwerp/Belgium': 3,
    'Presidio/Texas/USA': 3, 'Charlotte/North Carolina/USA': 6, 'Moscow/Russia': 9,
    'Abba/Saudi Arabia': 4, 'Hobby Airport/Houston/Texas/USA': 4, 'Tehran/Iran': 6,
    'Yemen': 1, 'Farmers Branch/Texas/USA': 3, 'Oasis Internet Cafe/Marrakech/Morocco': 3,
    'Lipetsk/Russia': 2, 'Piedras Negras': 2, 'Paris/France': 10,
    'Heathrow Airport/London/UK': 10, 'Buenos Aires/Argentina': 6,
    'Longhorn Motel/Carrizo Springs/Texas/USA': 3, 'Euless/Texas/USA': 2,
    'Cartagena/Columbia/USA': 5, 'Camden/New Jersey/USA': 3,
    'Kansas City/Missouri/USA': 6, 'the Argosy Riverside Casino/Kansas City/Missouri/USA': 3,
    'London Heathrow/UK': 10, 'Roanoke/Virginia/USA': 3,
    'Amsterdam/Netherlands': 3, 'Eagle Pass/USA': 2,
    'Ferndale/Michigan/USA': 3, 'Kansas City/USA': 5,
    'Ciudad Madero/Mexico': 3, 'Reston/Virginia/USA': 3,
    'Catalan Hotel/Madrid/Spain': 3, 'Chitral/Pakistan': 3,
    'Baltimore/Maryland/USA': 5, 'Gwadar/Pakistan': 3, 'Ecuador': 1,
    'Santa Domingo/Dominican Republic': 5, 'Saratov/Russia': 2, 'Syria': 1,
    'Maryland/USA': 3, 'Luxor Hotel/Cairo/Egypt': 3,
    'Columbus/Ohio/USA': 5
}



person_alias_map = {
    "Abdillah Zinedine": ["Abu Hafs", "Mehdi Rafiki"],
    "Saeed Hasham": ["Hamid Qatada", "Yasir Salman", "Saeed al Nami", "S. Hasham"],
    "Rifai Qasim": ["Omar Blakely", "Rifai", "R. Qasim"],
    "Satam Derwish": ["Ronald", "S. Derwish"],
    "Raeed Beandali": ["Ralph Bean", "R. Beandali"],
    "Reginald Cooper": ["Mahmud al-Dahab", "R. Cooper"],
    "Shadi abu Hoshar": ["Mousa Salah", "S. Abu Hoshar"],
    "Riyad Said": ["Jose Ramirez", "Riyad Yasser", "R. Said"],
    "Ali al-Omari": ["Ali"],
    "Abdul al Nami": ["Ali", "A. Nami", "Abdul Nami"],
    "Riduan Sungkar": ["Omar Evertts", "R. Sungkar"],
    "Abu Somad": ["A. Somad", "Somad"],
    "Yazid Bafaba": ["Y. Bafaba", "Bafaba"],
    "Jamal al Quso": ["J. al Quso"],
    "Adnan Hijazi": ["A. H.", "A Hijazi", "Hijazi"],
    "Carlos Morales": ["C. Morales"],
    "Imad Dahdah": ["I. Dahdah"],
    "Ahmed Doha": ["A. Doha"],
    "Abu Ayyash": ["A. Ayyash"],
    "Khalid Shamrani": ["K. Shamrani"],
    "Jamal al Shibh": ["J. al Shibh"],
    "Mamoun Zammar": ["M. Zammar"],
    "Wali Shakur": ["W. Shakur"],
    "Hamza al Midhar": ["H. al Midhar"],
    "Mounir Mzoudi": ["M. Mzoudi"],
    "Muhammed Dandani": ["M. Dandani"],
    "Said Khadr": ["S. Khadr"],
}

## Normalizing Location attribute

In [ ]:
def normalize_location_string(loc):
    loc = loc.strip().strip('/')
    parts = [p.strip() for p in loc.split('/')] if loc else []
    street_keywords = ['st', 'ave', 'road', 'rd', 'blvd', 'lane', 'court', 'drive', 'apt']
    if parts and (any(char.isdigit() for char in parts[0]) or any(kw in parts[0].lower() for kw in street_keywords)):
        parts = parts[1:]
    filtered_parts = [part for part in parts if part]
    return '/'.join(filtered_parts) if filtered_parts else np.nan

for record in json_data:
    places_raw = record.get('PLACES', '')
    place_list = [p.strip() for p in places_raw if p.strip()]
    normalized_places = [normalize_location_string(p) for p in place_list]
    normalized_places = [p for p in normalized_places if isinstance(p, str)]
    record['places'] = normalized_places

In [ ]:
def normalize_place(place):
    # Lowercase, strip spaces, remove trailing slashes
    return place.strip().lower().strip('/')

def get_best_place_score(place_str, location_ranking):
    place_norm = normalize_place(place_str)
    # Normalize all keys in location_ranking upfront (do once outside if possible for efficiency)
    normalized_ranking = {normalize_place(k): v for k, v in location_ranking.items()}

    # Try exact match on normalized full place string
    if place_norm in normalized_ranking:
        return normalized_ranking[place_norm]

    # Split place string into parts by common separators and normalize
    parts = [normalize_place(part) for part in re.split(r'[\\/,-]', place_str)]

    # Try longest segments first (descending by length)
    sorted_parts = sorted(parts, key=len, reverse=True)
    for part in sorted_parts:
        if part in normalized_ranking:
            return normalized_ranking[part]

    # If no match found, return 0
    return 0

reports = json_data

results = []
for report in reports:
    rid = report.get('ID', '')
    places = report.get('places', [])
    description = report.get('REPORTDESCRIPTION', '')

    place_scores = [get_best_place_score(p, location_importance_rank) for p in places]
    max_place_score = max(place_scores) if place_scores else 0

    desc_words = re.findall(r'\b\w+\b', description.lower())
    '''Sum/Count-Weighted: Take the sum of all danger word scores (or sum divided by total danger word occurrences, not all words).
    This signals if a report is densely populated with threat terms.'''
    danger_scores = [danger_word_ranking.get(word, 0) for word in desc_words if word in danger_word_ranking]
    avg_danger_score = sum(danger_scores)/len(danger_scores) if danger_scores else 0

    w1, w2 = 0.35, 0.65
    threat_score = w1 * max_place_score + w2 * avg_danger_score

    results.append({
        'ReportID': rid,
        'MaxPlaceScore': max_place_score,
        'AverageDangerScore': avg_danger_score,
        'ThreatScore': threat_score
    })

df = pd.DataFrame(results)

# Report Level Threat Score





In [ ]:
df

,ReportID,MaxPlaceScore,AverageDangerScore,ThreatScore
0,FBI_1,10,3.0,5.450
1,FBI_2,10,4.0,6.100
2,FBI_3,4,3.2,3.480
3,FBI_4,7,3.0,4.400
4,FBI_5,7,3.0,4.400
...,...,...,...,...
106,DIA_1,4,4.5,4.325
107,DIA_2,3,4.0,3.650
108,DIA_3,8,3.0,4.750
109,USCBP_1,4,3.0,3.350


For intelligence analysis and threat evaluation, the ranking of places by importance should reflect the following logic without code:

Highest Importance (Rank 10): Specific high-threat or high-value urban areas, cities, or known sensitive locations where intelligence activities, attacks, or operations are likely concentrated. Examples: New York City, Washington D.C., London, Paris, Moscow, Riyadh, Karachi.

Very High Importance (Rank 9): Major metropolitan regions or state capitals with significant strategic, political, or military importance. Examples: Los Angeles, Madrid, Cairo.

High Importance (Rank 8): Large cities or regions within countries that have recurrent intelligence interest or suspected risk clusters; also key transit hubs or economic centers. Examples: Houston, Chicago, Mexico City.

Moderate Importance (Rank 7): Smaller cities and towns known for specific intelligence leads or emerging threat indicators. Examples: Montreal, Panama City, Islamabad.

Notable Areas (Rank 6): Regional centers or places identified in reports but with limited contextual threat weight individually. Examples: Casablanca, Bucaramanga, Havana.

General Regional or Country Level (Rank 5 or below): Broader national or regional areas cited in reports without specific high-risk locality indicated. Use lower ranks here unless special conditions apply. Examples: Afghanistan, Cuba, Bahamas.

Generic or Minor Locations (Rank 3-4): Locations with little individual threat relevance or just supporting presence in reports, such as minor towns, or street level references. These rank lowest but are still recorded for completeness.

In summary, place importance should prioritize:

Localized threat hotspots over broad geopolitical regions.

Cities and areas with known or suspected intelligence activity.

Strategic, political, and economic centers over generic country references.

## Aggregating Reports ID's

In [ ]:
all_persons = set()

# Map each person to a set of file IDs (including references)
person_files = {}

for report in reports:
    file_id = report['ID']
    # Parse references; split on ';', ignore empty
    references = [ref for ref in report['REFERENCEID'].split(';') if ref] if report['REFERENCEID'] else []
    for person in report['PERSONS']:
        all_persons.add(person)
        if person not in person_files:
            person_files[person] = set()
        person_files[person].add(file_id)
        # Add reference files to this person
        for ref in references:
            person_files[person].add(ref)

# Convert sets to lists for final output
all_persons = list(all_persons)

for person in person_files:
    person_files[person] = list(person_files[person])

## Aggregating reports based on aliases

In [ ]:
aliased_list = defaultdict(dict)
skp_lst = set()

for person, aliases in person_alias_map.items():
    if person not in skp_lst:
        aliased_list[person]['files'] = list(person_files.get(person, []))
        aliased_list[person]['alias'] = []
        skp_lst.add(person)

        for alias in aliases:
            if alias not in skp_lst:
                aliased_list[person]['files'].extend(person_files.get(alias, []))
                aliased_list[person]['alias'].append(alias)
                skp_lst.add(alias)

        # Remove duplicates while preserving order
        seen = set()
        unique_files = []
        for f in aliased_list[person]['files']:
            if f not in seen:
                unique_files.append(f)
                seen.add(f)
        aliased_list[person]['files'] = unique_files

In [ ]:
aliased_list

## Processing Date
To organize in ascending order

In [ ]:
records = json_data

def clean_date_string(date_str):
    today = datetime.today()
    parts = date_str.strip().split('/')

    # Pad the list to ensure [month, day, year]
    while len(parts) < 3:
        parts.append('')

    # Fill missing values
    m = int(parts[0]) if parts[0].strip() else 1
    d = int(parts[1]) if parts[1].strip() else 1
    y = int(parts[2]) if parts[2].strip() else today.year

    # If only blanks, use today's date
    if m == 1 and d == 1 and (not parts[2].strip()):
        return today

    try:
        return datetime(year=y, month=m, day=d)
    except ValueError:
        return today

# Build id_to_date with cleaning
id_to_date = {}
for record in records:
    date_str = record.get('REPORTDATE', '').strip()
    try:
        if date_str:
            date_obj = clean_date_string(date_str)
        else:
            date_obj = datetime.today()
        id_to_date[record['ID']] = date_obj
    except Exception as e:
        id_to_date[record['ID']] = datetime.today()  # fallback
        print(f"Error parsing {date_str}: {e}")

for name, info in aliased_list.items():
    file_date_pairs = [(file, id_to_date.get(file)) for file in info['files']]
    file_date_pairs.sort(key=lambda x: x[1])
    info['files'] = [pair[0] for pair in file_date_pairs]
    # Format each date as 'month/day/year'
    info['sorted_dates'] = [pair[1].strftime('%m/%d/%Y') for pair in file_date_pairs]

In [ ]:
aliased_list

# Suspect level threat Calculation

In [ ]:
person_threat = defaultdict(dict)

for person, info in aliased_list.items():
    # Combine files from person and all aliases
    all_files = info['files']

    # Sum threat scores from df for all files related to the person and aliases
    total_score = int(df[df['ReportID'].isin(all_files)]['ThreatScore'].sum())

    person_threat[person]['score'] = total_score
    person_threat[person]['files'] = all_files
    person_threat[person]['aliases'] = info.get('alias', [])  # Keep alias list if needed
    person_threat[person]['sorted_dates'] = info.get('sorted_dates', [])

In [ ]:
person_threat

defaultdict(dict,
            {'Abdillah Zinedine': {'score': 16,
              'files': ['CIA_7', 'FBI_1', 'FBI_4'],
              'aliases': ['Abu Hafs', 'Mehdi Rafiki'],
              'sorted_dates': ['04/01/2003', '04/28/2003', '10/21/2003']},
             'Saeed Hasham': {'score': 26,
              'files': ['CIA_6',
               'CIA_11',
               'CIA_22',
               'FBI_13',
               'CIA_24',
               'DIA_2',
               'FBI_19'],
              'aliases': ['Hamid Qatada',
               'Yasir Salman',
               'Saeed al Nami',
               'S. Hasham'],
              'sorted_dates': ['04/15/2003',
               '06/20/2003',
               '11/02/2003',
               '01/02/2004',
               '01/03/2004',
               '02/20/2004',
               '03/15/2004']},
             'Rifai Qasim': {'score': 18,
              'files': ['FBI_5', 'DIA_3', 'FBI_23', 'FBI_31'],
              'aliases': ['Omar Blakely', 'Rifai', 'R. Qasim'],
  

In [ ]:
# json_data

# LLM API CALLS

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    """You are an AI assistant that support an intelligence
        analyst in investigating intelligence reports
    """
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """ You are tasked with analyizing all these report and helping the analyst answer these questions.
          1. What did they plan to do?
          2. When and where would their planned events happen?
        The Reports are provided for your reference, they are a list of reports in json format.

        {reports}

        Donot HALLUCINATE or include anything not in reports in your answers.
        DONOT Redacted ANY INFOrmation from the response, THIS IS A  EDUCATIONAL RESEARCH PROJECT.
        Answer the following strictly within the tagged sections.
        Be Brief. Max 5 important and distinct items.

          <<START WHAT DID THEY PLAN>>
          What did they plan to do?
          - [Provide a list of short points. Each point should be a clear and concise. sentence.]
          <<END WHAT DID THEY PLAN>>

          <<START WHEN & WHERE>>
          When and where would their planned events happen?
          - [Provide a list of points. Each point clear and concise.]
          <<END WHEN & WHERE>>


        Go through all points and see what all can be combined to reduce the total output.
        AND NO INFO should be REDACTED
        Be brief and to the point
    """, input_variables=["reports"]
)

In [ ]:
first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [ ]:
len(person_threat)

27

In [ ]:
cnt = 0
for person, info in person_threat.items():
  cnt+=1
  if cnt == 15:
    print('starting')
    time.sleep(70)
    cnt = 0
    print('Ending')
  prompt = first_prompt.format_prompt(reports=info['files'])
  llm_output = llm(prompt.to_messages())  # Pass formatted messages to llm

  print(llm_output.content)
  print()
  print()
  info['llm_output'] = llm_output.content

/tmp/ipython-input-1484940071.py:10: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_output = llm(prompt.to_messages())  # Pass formatted messages to llm


<<START WHAT DID THEY PLAN>>
What did they plan to do?
- Conduct a bombing attack.
- Assassinate a political figure.
- Engage in armed robbery.
- Distribute propaganda.
- Incite civil unrest.
<<END WHAT DID THEY PLAN>>

<<START WHEN & WHERE>>
When and where would their planned events happen?
- In the vicinity of the U.S. Capitol.
- During a public demonstration.
- In a specific city (location not explicitly stated but implied to be within the US).
- At a government building.
- On a specific date (not explicitly stated but implied to be in the near future).
<<END WHEN & WHERE>>


<<START WHAT DID THEY PLAN>>
What did they plan to do?
- Conduct a bombing attack.
- Assassinate a political figure.
- Incite civil unrest and riots.
- Disrupt critical infrastructure.
- Acquire weapons and explosives.
<<END WHAT DID THEY PLAN>>

<<START WHEN & WHERE>>
When and where would their planned events happen?
- The bombing was planned for a specific date in the near future.
- The assassination target w

In [ ]:
person_threat

defaultdict(dict,
            {'Abdillah Zinedine': {'score': 16,
              'files': ['CIA_7', 'FBI_1', 'FBI_4'],
              'aliases': ['Abu Hafs', 'Mehdi Rafiki'],
              'sorted_dates': ['04/01/2003', '04/28/2003', '10/21/2003'],
              'llm_output': '<<START WHAT DID THEY PLAN>>\nWhat did they plan to do?\n- Conduct a bombing attack.\n- Assassinate a political figure.\n- Engage in armed robbery.\n- Distribute propaganda.\n- Incite civil unrest.\n<<END WHAT DID THEY PLAN>>\n\n<<START WHEN & WHERE>>\nWhen and where would their planned events happen?\n- In the vicinity of the U.S. Capitol.\n- During a public demonstration.\n- In a specific city (location not explicitly stated but implied to be within the US).\n- At a government building.\n- On a specific date (not explicitly stated but implied to be in the near future).\n<<END WHEN & WHERE>>'},
             'Saeed Hasham': {'score': 26,
              'files': ['CIA_6',
               'CIA_11',
               'CIA_

## Extracting what, when and where

In [ ]:
def extract_section(text, start_marker, end_marker):
    pattern = rf"{start_marker}(.*?){end_marker}"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

def split_points(text):
    # Split on lines starting with dash or number + dot, ignore empty lines
    lines = [line.strip() for line in re.split(r'\n\s*[-\d]+\.\s*|\n\s*-\s*', text) if line.strip()]
    return lines

for person, info in person_threat.items():
    llm_text = info.get('llm_output', '')
    what_text = extract_section(llm_text, "<<START WHAT DID THEY PLAN>>", "<<END WHAT DID THEY PLAN>>")
    when_where_text = extract_section(llm_text, "<<START WHEN & WHERE>>", "<<END WHEN & WHERE>>")

    info['what'] = split_points(what_text)
    info['when & where'] = split_points(when_where_text)


In [ ]:
person_threat

defaultdict(dict,
            {'Abdillah Zinedine': {'score': 16,
              'files': ['CIA_7', 'FBI_1', 'FBI_4'],
              'aliases': ['Abu Hafs', 'Mehdi Rafiki'],
              'sorted_dates': ['04/01/2003', '04/28/2003', '10/21/2003'],
              'llm_output': '<<START WHAT DID THEY PLAN>>\nWhat did they plan to do?\n- Conduct a bombing attack.\n- Assassinate a political figure.\n- Engage in armed robbery.\n- Distribute propaganda.\n- Incite civil unrest.\n<<END WHAT DID THEY PLAN>>\n\n<<START WHEN & WHERE>>\nWhen and where would their planned events happen?\n- In the vicinity of the U.S. Capitol.\n- During a public demonstration.\n- In a specific city (location not explicitly stated but implied to be within the US).\n- At a government building.\n- On a specific date (not explicitly stated but implied to be in the near future).\n<<END WHEN & WHERE>>',
              'what': ['What did they plan to do?',
               'Conduct a bombing attack.',
               'Assassinat

# Storing the Results

In [ ]:
regular_dict = dict(person_threat)

# Save to a JSON file
with open('output_data.json', 'w') as f:
    json.dump(regular_dict, f, indent=4)

In [ ]:
with open('input_data.json', 'w') as f:
    json.dump(json_data, f, indent=4)

In [ ]:
# https://ai.google.dev/gemini-api/docs/rate-limits#free-tier

In [ ]:
# # Read the file
# with open('/content/drive/MyDrive/dataset.txt', 'r', encoding='utf-8') as f:
#     content = f.read()

# # Split reports
# reports = re.split(r'\nREPORT\n', content)

# # Prepare csv
# header = ['ID', 'REPORTDATE', 'REFERENCEID', 'REPORTSOURCE', 'REPORTDESCRIPTION', 'PERSONS', 'DATES', 'PLACES', 'ORGANIZATIONS']
# rows = []
# for rep in reports:
#     if not rep.strip():
#         continue
#     id_ = re.search(r'ID: ([^\n]*)', rep)
#     date = re.search(r'REPORTDATE: ([^\n]*)', rep)
#     refid = re.search(r'REFERENCEID: ([^\n]*)', rep)
#     src = re.search(r'REPORTSOURCE: ([^\n]*)', rep)
#     des = re.search(r'REPORTDESCRIPTION: ([^\n]*)', rep)
#     persons = re.search(r'PERSONS: ([^\n]*)', rep)
#     dates = re.search(r'DATES: ([^\n]*)', rep)
#     places = re.search(r'PLACES: ([^\n]*)', rep)
#     orgs = re.search(r'ORGANIZATIONS: ([^\n]*)', rep)
#     row = [
#         id_.group(1).strip() if id_ else '',
#         date.group(1).strip() if date else '',
#         refid.group(1).strip() if refid else '',
#         src.group(1).strip() if src else '',
#         des.group(1).strip() if des else '',
#         persons.group(1).strip() if persons else '',
#         dates.group(1).strip() if dates else '',
#         places.group(1).strip() if places else '',
#         orgs.group(1).strip() if orgs else ''
#     ]
#     rows.append(row)
